# Extract Proteins from Blastp

A blastp returned a list of contigs that matched known 16s rRNA proteins. To create a phylogenetic tree, I had to determine the protein sequence of these contigs, by looking back to the ORF file. This script creates a FASTA file with the protein sequences of the contigs in the blast file.

In [1]:
# import pandas, a data-wrangling library
import pandas as pd
from Bio import SeqIO
import textwrap

### I. Make pandas dataframe from blastp file

In [2]:
blastp_file = "16s_protein_vs_ERR599031_ORFs.blastp"

blastp_headers = ["query_seq_name",
                  "db_seq_name",
                  "percent_identity",
                  "alignment_len",
                  "num_mismatches",
                  "num_gaps",
                  "query_start",
                  "query_end",
                  "subject_start",
                  "subject_end",
                  "e-value",
                  "bitscore"]

blastp_df = pd.read_csv(
    blastp_file, header=None,
    delimiter="\t", names=blastp_headers)

In [3]:
# Sort by evalue
blastp_df.sort_values('e-value', inplace=True)

In [4]:
# head() displays the first n rows of the dataframe
blastp_df.head(n=10)

,query_seq_name,db_seq_name,percent_identity,alignment_len,num_mismatches,num_gaps,query_start,query_end,subject_start,subject_end,e-value,bitscore
54,RSMB_ECOLI/239-427,c_000000001032_5,56.545,191,79,2,3,189,245,435,6.230000e-68,214.0
36,RSMB_HAEIN/251-447,c_000000001032_5,53.535,198,86,2,1,197,243,435,2.140000e-63,202.0
18,RSMB_COXBU/231-427,c_000000001032_5,46.429,196,101,2,1,196,243,434,2.150000e-52,174.0
19,RSMB_COXBU/231-427,c_000000006632_1,45.355,183,95,3,12,194,163,340,6.760000e-44,149.0
55,RSMB_ECOLI/239-427,c_000000006632_1,45.652,184,95,3,9,188,160,342,9.020000e-44,149.0
37,RSMB_HAEIN/251-447,c_000000006632_1,44.920,187,94,5,12,196,163,342,8.940000e-43,147.0
20,RSMB_COXBU/231-427,c_000000048563_1,62.857,105,36,2,3,107,57,158,4.610000e-38,129.0
0,RSMB_SYNY3/245-444,c_000000001032_5,39.698,199,113,2,1,199,243,434,7.670000e-36,130.0
1,RSMB_SYNY3/245-444,c_000000000008_7,33.166,199,127,2,2,200,250,442,9.120000e-36,130.0
21,RSMB_COXBU/231-427,c_000000059173_1,44.186,129,71,1,66,194,2,129,1.020000e-35,122.0


### Find protein sequences in ORF file, to create FASTA file

In [5]:
ORF_file = "ERR599031_ORFs.faa"
outfile = "16s_ORF_project.fasta"

In [6]:
# Use biopython to parse full ORF file
fasta_sequences = SeqIO.parse(open(ORF_file),'fasta')

In [7]:
# Iterate over entries in ORF fasta file,
# writing to new file any entry contained in blast file

with open(outfile, 'w') as file:
    
    for fasta in fasta_sequences:
        if blastp_df['db_seq_name'].str.contains(fasta.id).any():
            print(">" + fasta.id)
            file.write(">" + fasta.id + "\n")

            print(textwrap.fill(str(fasta.seq), width=60))
            file.write(textwrap.fill(str(fasta.seq), width=60))
            file.write("\n")

>c_000000000008_7
MANKLSAREHTVRILDKFDRSFNRLSTIFENYFASNETSFKSRPEITHLVQDIIRHRGYI
DHIIISLFRGNYKTADNTLRNILRLGTYEILYRDQVPDFAAVNEAVELAKRKRGKGAAGL
GNALLRKIRKADTSVEKALSPETPIHQMAALLSHPGWMISRWIDSLGWERTIQFCEWNNT
VPILTVRMNSLQTEKEAFEKFLEESMVQWEPKQIIPEFYTVHRVSKIRSSIQFNEGHFSF
QDVSSGLVTNLLKPTEGDSVLDVCAAPGGKCGYMAELMRNSGSIYAFDVDADRHALLKES
MKRLKLNSVNVEQKDATADSFQVSDKILIDAPCTGTGVMAKRADLKWRRRPSDLDEMVSV
QKRIINHIAQFLESEGELVYATCSVEPEENWGVVDSFLADNPSFSVQSAEGRVPSRFLDN
MGALSTFPPEHGMDGVFAVILRRN
>c_000000000020_1
MKSFNSRLSGIIQLKESHLCVGLDMNPEELGKAEVSIEDLKSHTYKVIDATRDLAAAFKP
NLAFFERWGSTGFKWLEETMEYFRDDVIVIGDGKR
>c_000000000020_11
MRDARFHAFQVIQGFYRYNIRLKFFRDQYFKKHSLSRNDLSRTLVLTNEVVRWQGRLDYW
ISLYLDKPINKLHPSVHNILRLGYFEALMDKSIPQHAVVHSWVELAKSEIGVQFGGLVNA
VLRKTEKINPTERFEDQSLNAWYSYPKWLFQKWNKQFGETKANQLCEYFNKPSPNDIRLN
GDKGSRLAIVNQLEKEGIELVPSPESDRFFRVKTGMVKVLNNNQFKTGLIHAQDCASGAV
VELLDPHPGENILDVCAAPGTKSGYIVEKMKGDGQVYSSDISQKRIELGNESTKALKLII
QWDCKDASKDDFPIADRILVDAPCTGTGVIGRRTDIRWRREEDDIKSLVEIQSAILNHMA
QFLKQDGILVYATCSIEAEENWNVVESFLK